# Scraping Top Repositories for Topics on GitHub

Introduction 
- Introduction about web scraping <br>
  `Web scraping` is the process of using bots to extract content and data from a website.
- Introduction about GitHub and the problem statement <br>
  `GitHub,Inc.` is a provider of Internet hosting for software development and version control using Git. It offers the distributed version control and source code management functionality of Git, plus its own features.
- Mention the tools you're using (Python, requests, Beautiful Soup, Pandas) <br>
  A) `Python` is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.<br>
  B) `Requests` is a HTTP library for the Python programming language. The goal of the project is to make HTTP requests simpler and more human-friendly.<br>
  C) `Beautiful Soup` is a Python package for parsing HTML and XML documents. It creates a parse tree for parsed pages that can be used to extract data from HTML, which is useful for web scraping.<br>
  D) `Pandas` is a software library written for the Python programming language for data manipulation and analysis.


Here are the steps we'll follow:

- We're going to scrape https://github.com/topics
- We'll get a list of topics. For each topic, we'll get topic title, topic page URL and topic description
- For each topic, we'll get the top 25 repositories in the topic from the topic page
- For each repository, we'll grab the repo name, username, stars and repo URL
- For each topic we'll create a CSV file in the following format:

```
Repo Name,Username,Stars,Repo URL
three.js,mrdoob,80800,https://github.com/mrdoob/three.js
libgdx,libgdx,19800,https://github.com/libgdx/libgdx
```

## Scrape the list of topics from Github

- use requests to downlaod the page
- user BS4 to parse and extract information
- convert to a Pandas dataframe

Let's write a function to download the page.

In [176]:
!pip install requests --upgrade --quiet
import requests
from bs4 import BeautifulSoup

# Function to get topics
def get_topics_page():
    #download the page
    topics_url = 'https://github.com/topics'
    response = requests.get(topics_url)
    #check successful response
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(topic_url))
    #parse using Beautiful soup
    doc = BeautifulSoup(response.text,'html.parser')
    return doc

In [177]:
doc = get_topics_page()

Let's create some helper functions to parse information from the page.

To get topic titles, we can pick `p` tags with the `class` ...


In [178]:
def get_topic_titles(doc):
    selection_class = 'f3 lh-condensed mb-0 mt-1 Link--primary'
    topic_title_tags = doc.find_all('p', {'class': selection_class})
    topic_titles = []
    for tag in topic_title_tags:
        topic_titles.append(tag.text)
    return topic_titles

get_topic_titles can be used to get the list of titles

In [179]:
titles = get_topic_titles(doc)

In [180]:
len(titles)

30

In [181]:
titles[:5]

['3D', 'Ajax', 'Algorithm', 'Amp', 'Android']

Similarly we have defined functions for descriptions and URLs.

In [182]:
def get_topic_descs(doc):
    desc_selector = 'f5 color-fg-muted mb-0 mt-1'
    topic_desc_tags = doc.find_all('p', {'class': desc_selector})
    topic_descs = []
    for tag in topic_desc_tags:
        topic_descs.append(tag.text.strip())
    return topic_descs

get_topic_descs can be used to get list of descriptions of titles

In [183]:
descs = get_topic_descs(doc)

In [184]:
len(descs)

30

In [185]:
descs[:5]

['3D modeling is the process of virtually developing the surface and structure of a 3D object.',
 'Ajax is a technique for creating interactive web applications.',
 'Algorithms are self-contained sequences that carry out a variety of tasks.',
 'Amp is a non-blocking concurrency library for PHP.',
 'Android is an operating system built by Google designed for mobile devices.']

In [186]:
def get_topic_urls(doc):
    topic_link_tags = doc.find_all('a', {'class': 'no-underline flex-grow-0'})
    topic_urls = []
    base_url = 'https://github.com'
    for tag in topic_link_tags:
        topic_urls.append(base_url + tag['href'])
    return topic_urls

get_topic_urls can be used to get the list of url of the titles

In [187]:
urls = get_topic_urls(doc)

In [188]:
len(urls)

30

In [189]:
urls[:5]

['https://github.com/topics/3d',
 'https://github.com/topics/ajax',
 'https://github.com/topics/algorithm',
 'https://github.com/topics/amphp',
 'https://github.com/topics/android']

Let's put this all together into a single function

In [190]:
import pandas as pd
def scrape_topics():
    topics_url = 'https://github.com/topics'
    response = requests.get(topics_url)
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(topic_url))
    doc = BeautifulSoup(response.text, 'html.parser')
    topics_dict = {
        'title': get_topic_titles(doc),
        'description': get_topic_descs(doc),
        'url': get_topic_urls(doc)
    }
    return pd.DataFrame(topics_dict)

In [191]:
scrape_topics()[:5]

,title,description,url
0,3D,3D modeling is the process of virtually develo...,https://github.com/topics/3d
1,Ajax,Ajax is a technique for creating interactive w...,https://github.com/topics/ajax
2,Algorithm,Algorithms are self-contained sequences that c...,https://github.com/topics/algorithm
3,Amp,Amp is a non-blocking concurrency library for ...,https://github.com/topics/amphp
4,Android,Android is an operating system built by Google...,https://github.com/topics/android


## Get the top repositories from a topic page

- use requests to downlaod the selected topic page
- user BS4 to parse and extract information
- convert to a Pandas dataframe


In [192]:
def get_topic_page(topic_url):
    # Download the page
    response = requests.get(topic_url)
    # Check successful response
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(topic_url))
    # Parse using Beautiful soup
    topic_doc = BeautifulSoup(response.text, 'html.parser')
    return topic_doc

In [193]:
topic_doc = get_topic_page('https://github.com/topics/3d')

In [194]:
h3_selection_class = 'f3 color-fg-muted text-normal lh-condensed'
repo_tags = topic_doc.find_all('h3', {'class': h3_selection_class} )

In [195]:
star_tags = topic_doc.find_all('span', { 'class': 'Counter js-social-count'})

In [196]:
def parse_star_count(stars_str):
    stars_str = stars_str.strip()
    if stars_str[-1] == 'k':
        return int(float(stars_str[:-1]) * 1000)
    return int(stars_str)

In [197]:
def get_repo_info(h3_tag, star_tag):
    # returns all the required info about a repository
    base_url = 'https://github.com'
    a_tags = h3_tag.find_all('a')
    username = a_tags[0].text.strip()
    repo_name = a_tags[1].text.strip()
    repo_url =  base_url + a_tags[1]['href']
    stars = parse_star_count(star_tag.text.strip())
    return username, repo_name, stars, repo_url

In [198]:
get_repo_info(repo_tags[0], star_tags[0])

('mrdoob', 'three.js', 80800, 'https://github.com/mrdoob/three.js')

In [199]:
def get_topic_repos(topic_doc):
    # Get the h3 tags containing repo title, repo URL and username
    h3_selection_class = 'f3 color-fg-muted text-normal lh-condensed'
    repo_tags = topic_doc.find_all('h3', {'class': h3_selection_class} )
    # Get star tags
    star_tags = topic_doc.find_all('span', { 'class': 'Counter js-social-count'})
    
    topic_repos_dict = { 'username': [], 'repo_name': [], 'stars': [],'repo_url': []}

    # Get repo info
    for i in range(len(repo_tags)):
        repo_info = get_repo_info(repo_tags[i], star_tags[i])
        topic_repos_dict['username'].append(repo_info[0])
        topic_repos_dict['repo_name'].append(repo_info[1])
        topic_repos_dict['stars'].append(repo_info[2])
        topic_repos_dict['repo_url'].append(repo_info[3])
        
    return pd.DataFrame(topic_repos_dict)

In [200]:
get_topic_repos(topic_doc)[:5]

,username,repo_name,stars,repo_url
0,mrdoob,three.js,80800,https://github.com/mrdoob/three.js
1,libgdx,libgdx,19800,https://github.com/libgdx/libgdx
2,pmndrs,react-three-fiber,17500,https://github.com/pmndrs/react-three-fiber
3,BabylonJS,Babylon.js,16300,https://github.com/BabylonJS/Babylon.js
4,aframevr,aframe,14000,https://github.com/aframevr/aframe


In [201]:
import os

def scrape_topic(topic_url, path):
    if os.path.exists(path):
        print("The file {} already exists. Skipping...".format(path))
        return
    topic_df = get_topic_repos(get_topic_page(topic_url))
    topic_df.to_csv(path, index = None)

## Putting it all together

- We have a funciton to get the list of topics
- We have a function to create a CSV file for scraped repos from a topics page
- Let's create a function to put them together

In [202]:
def scrape_topics_repos():
    print('Scraping list of topics')
    topics_df = scrape_topics()
    
    os.makedirs('data', exist_ok=True)
    for index, row in topics_df.iterrows():
        print('Scraping top repositories for "{}"'.format(row['title']))
        scrape_topic(row['url'], 'data/{}.csv'.format(row['title']))

Let's run it to scrape the top repos for the all the topics on the first page of https://github.com/topics

In [203]:
scrape_topics_repos()

Scraping list of topics
Scraping top repositories for "3D"
The file data/3D.csv already exists. Skipping...
Scraping top repositories for "Ajax"
The file data/Ajax.csv already exists. Skipping...
Scraping top repositories for "Algorithm"
The file data/Algorithm.csv already exists. Skipping...
Scraping top repositories for "Amp"
The file data/Amp.csv already exists. Skipping...
Scraping top repositories for "Android"
The file data/Android.csv already exists. Skipping...
Scraping top repositories for "Angular"
The file data/Angular.csv already exists. Skipping...
Scraping top repositories for "Ansible"
The file data/Ansible.csv already exists. Skipping...
Scraping top repositories for "API"
The file data/API.csv already exists. Skipping...
Scraping top repositories for "Arduino"
The file data/Arduino.csv already exists. Skipping...
Scraping top repositories for "ASP.NET"
The file data/ASP.NET.csv already exists. Skipping...
Scraping top repositories for "Atom"
The file data/Atom.csv alre

We can check that the CSVs were created properly

## References

Summary

- Searched and Collected the top topics of Github into a CSV file.
- Searched and Collected the topic repositories and their details of each topic into a CSV file


References to links you found useful

- https://docs.python-requests.org/en/latest/
- https://www.geeksforgeeks.org/implementing-web-scraping-python-beautiful-soup/
